### Chapter 13 - Custom User Model:

The biggest problem with the auth.user is that it is very hard to modify once you migrate, also in django documentation it is said that we should avoid any changes.<br>

The solution is to use a CustomUser model.<br>
Please understand that in order to use a CustomUser model you should be in the early stage of you project, meaning that you shouldnt have done any migrations otherwise it is almost imposible to achieve.<br>

So, lets start with a new project:

### BookStore:
1. Create a new django project called 'BookStore'
2. Add a new app called 'accounts' to the project
3. Add the new add to the INSTALLED_APP list (settings.py)
4. Navigate to account.models.py
5. From django.contrib.auth.models import AbstractUser
6. Create a new model called CustomUser and extend it from AbstractUser class

In [ ]:
from django.contrib.auth.models import AbstractUser

class CustomUser(AbstractUser):
    pass

Now we need to tell django that it should use this model instead of its existing auth model

7. Navigate to settings.py and add a new variable called AUTH_USER_MODEL and pass 'accounts.CustomUser' to it

In [ ]:
AUTH_USER_MODEL = 'accounts.CustomUser'

Lets talk about AbstractUser Class:

The abstract user class is a class that already has some fields in it, you can use it and also add more fields to it.

Here are the fields included in the AbstractUser class:

- username: A string field for the username, which is unique.
- first_name: A string field for the user's first name.
- last_name: A string field for the user's last name.
- email: An email field for the user's email address.
- password: A string field for storing the hashed password.
- groups: A many-to-many relationship to the Group model, representing the groups the user belongs to.
- user_permissions: A many-to-many relationship to the Permission model, representing the permissions granted to the user.
- is_staff: A boolean field indicating whether the user can access the admin site.
- is_active: A boolean field indicating whether the user account is active.
- is_superuser: A boolean field indicating whether the user has all permissions without explicitly assigning them.
- last_login: A datetime field for the last time the user logged in.
- date_joined: A datetime field for when the user account was created.

Now lets add a new field to it just for seeing how it is:

In [ ]:
from django.db import models
from django.contrib.auth.models import AbstractUser

class CustomUser(AbstractUser):
    age = models.PositiveIntegerField()

PositiveIntegerField only accepts Possitive values

Before we mode on, lets talk about the difference between AbstractUser and AbstractBaseUser classes:


### BastractUser: 

In Django, both AbstractUser and AbstractBaseUser are classes used to create custom user models, but they serve different purposes and offer different levels of customization. Here’s a detailed comparison of the two:
AbstractUser

- Purpose:

    - AbstractUser is a more complete and feature-rich base class for creating custom user models.
    - It provides all the fields and methods commonly needed for a user model out of the box.

- Fields:

    - Includes common fields such as username, first_name, last_name, email, password, groups, user_permissions, is_staff, is_active, is_superuser, last_login, and date_joined.

- Use Case:

    - Use AbstractUser if you need to extend the default Django user model with additional fields or methods, but want to retain all the built-in functionality.

### AbstractBaseUser

- Purpose:

    - AbstractBaseUser provides the bare minimum for a custom user model, offering a lot more flexibility for defining a completely custom user model.
    - It includes only the core authentication functionality, without any of the fields or methods provided by AbstractUser.

- Fields:

    - Includes only the password, last_login, and is_active fields.
    - Requires you to define additional fields such as username or email, and manage user authentication and permissions more manually.

- Use Case:

    - Use AbstractBaseUser if you need to build a custom user model from scratch, with completely different fields and authentication logic than what is provided by Django’s default user model.

### Now lets add some constraints to our model:

In [ ]:
from django.db import models
from django.contrib.auth.models import AbstractUser

class CustomUser(AbstractUser):
    age = models.PositiveIntegerField(null=True, blank=True) 

In Django, null and blank are two model field options that control different aspects of data validation and database schema.<br>

Null is for database and none is for the form that user is going to fillin.

Please understand that for a charfield a "" is considered blank but not null.

### CustomUser Forms:

Now that we created our CustomUser-class we need to work on its form! Why? because if we don't django will use its own predefined form.<br>
The predefiened form will be used by django in both adminstrative area and signups. We obviously don't want that and we want our own forms to be desiplayed. This we can style the admin area in the way that we want.<br>

In general, what we are oing to do is to extend the predefined forms and modify them:<br>
What you need to understand here is that there are two forms that we are going to extend and modify, 'UserCreationForm' and 'UseChangeForm', the first form is used when we want to create (signup) a new user and the second one is for adminstrative area.<be>
<br>
1. Create a file called forms.py in your accounts app.
2. Create a class called CustomCreationForm inside it
3. From django.contrib.auth.forms import UserCreationForm
4. Extend your class from UserCreationForm
5. Create a Meta class inside it to modify its behaviour
6. Create a class-attribute inside your Meta class called 'model' and pass your CustomModel to it.
7. Createa another class-attribute called fields and pass '__all__'
- This will add the age field we added to our AbstractUser class to our form class.

In [ ]:
from django.contrib.auth.forms import UserCreationForm, UserChangeForm
from .models import CustomUser

class CustomUserCreationForm(UserCreationForm):
    class Meta:
        model = CustomUser
        fields = '__all__'

Now, lets extend and modify our UserChangeForm:

8. Create a new class called CustomUserChangeForm and extend it from UserChangeForm
9. Create a Meta class inside it and pass the model exactly like what you did for the CustomUserCreationForm class.

In [ ]:
class CustomUserChangeForm(UserChangeForm):
    class Meta:
        model = CustomUser
        fields = '__all__'

Sofar we have created our model and its forms. We have already informed django about the model but the thing is that django is not aware of the forms that we have created. To fix this:<br>
<br>

1. Navigate to accounts.admins.py
2. From django.contrib.auth.admin import UserAdmin
3. Create a new class called 'CustomUserAdmin' and extend it from UserAdmin
4. Create a class-attribute called add_form and pass the CustomUserCreationForm to it (you need to import it first)
5. Create another class-attribute called form and pass the CustomUserChangeForm to it
6. Also, you need to add a third class-attribute called model and pass the CustomUser model to it.
7. Don't forget to register your model.

Your accounts.admin.py file should look like the following block:

In [ ]:
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin
from .forms import CustomUserCreationForm, CustomUserChangeForm
from .models import CustomUser


class CustomUserAdmin(UserAdmin):
    add_form = CustomUserCreationForm
    form = CustomUserChangeForm
    model = CustomUser


admin.site.register(CustomUser, CustomUserAdmin)

Now, django knows what forms and models it has to use.<br>

Now, we can use the makemigration and migrate command.<br>

But, you should undertsand that you need to them only for your accounts app! and not the whole project.<br>

In [ ]:
python manage.py makemigrations accounts

python manage.py migrate

Now, you need to create a super-user and run the server

In [ ]:
python manage.py createsuperuser

python manage.py runserver

If you visit the admin page, login, and check the users table, you will notice that the 'age' column is not added! to fix this:<br>

1. Navigate to accounts.admin.py and add a new class-attribute called fieldssets and pass 'UserAdmin.fieldsets + ((None, {'fields':('age',)}))

In [ ]:
class CustomUserAdmin(UserAdmin):
    add_form = CustomUserCreationForm
    form = CustomUserChangeForm
    model = CustomUser
    fieldsets = UserAdmin.fieldsets + (
        (None, {'fields': ('age',)}),
    )

Now if you save and reload the page you will see that the age is added at the end of your page.<br>

There is a problem! if you try to add a new user using the adminstrative area you will see that the form doesn not have the age field, to fix this:<br>
add the following to your CustomUserAdmin class:

In [ ]:
add_fieldsets = UserAdmin.add_fieldsets + (
        (None, {'fields': ('age',)}),
    )

In the next notebook we will continue working on our project